### **Task 2: Sentiment and Thematic Analysis**
- Description: Quantify review sentiment and identify themes to uncover satisfaction drivers and pain points.

In [1]:
import os 
import sys

system_path = os.path.abspath("..")
if system_path not in sys.path:
    sys.path.append(system_path)

%load_ext autoreload
%autoreload 2
from scripts.sentiment.sentiment_analysis import SentimentAnalyzer
from scripts.sentiment.thematic_analysis import ThematicAnalyzer

c:\Users\jonep\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'bertopic'

In [4]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

inputs = tokenizer("Hello, my dog isn't happy", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'NEGATIVE'

In [5]:
inputs

{'input_ids': tensor([[ 101, 7592, 1010, 2026, 3899, 3475, 1005, 1056, 3407,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}